In [1]:
# Load packages
import sys
sys.path.append('..')
from typing import Tuple
import numpy as np
from classes.universe import Universe
from classes.helper_functions.helpers import get_spatial_neighbours, get_vertices_in_slice
import os

In [2]:
def pick_vertex_dense(universe: Universe):
    """
    Picks a vertex in the universe at a timeslice with the maximum spatial volume.
    """
    slice_sizes = universe.slice_sizes
    vertices_in_slice = get_vertices_in_slice(universe)
    
    # Find which timeslice has the maximum spatial volume
    max_spatial_volume_timeslice = np.argmax(slice_sizes)
    
    # Pick a random vertex
    vertices = vertices_in_slice[max_spatial_volume_timeslice]
    vertex = np.random.choice(vertices)

    return vertex

def measure_spectral_dimension(
        universe: Universe,
        sigma_max: int,
        diffusion_constant: float,
        only_spatial: bool,
        start_flag: int,
        T: int,
        k0: float,
        target_volume: int,
        thermal_sweeps: int,
        main_sweeps: int,
        chain: int,
        run: int
        ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Measure the spectral dimension of the universe.

    Args:
        universe (Universe): Universe object
        sigma_max (int): Maximum sigma value
        diffusion_constant (float): Diffusion constant
        only_spatial (bool): If True, only consider spatial neighbors
        start_flag (int): 0 if start vertex should be first vertex, 1 if start vertex should be picked randomly, \
            2 if start vertex should be picked randomly from the densest timeslice
        T (int): Number of timeslices
        k0 (float): Coupling constant
        target_volume (int): Target volume
        thermal_sweeps (int): Number of thermalization sweeps
        main_sweeps (int): Number of main sweeps
        chain (int): Chain number
        run (int): Run number

    Returns:
        prob (np.ndarray): Array of probabilities
        spec_dim (np.ndarray): Array of spectral dimensions
        spec_dim1 (np.ndarray): Array of spectral dimensions
    """
    # Initialize variables
    if only_spatial:
        connectivity = get_spatial_neighbours(universe)
    else:
        connectivity = universe.vertex_neighbours

    if start_flag == 0:
        start = connectivity.keys()[0]
    elif start_flag == 1:
        start = np.random.choice(list(connectivity.keys()))
    else:
        start = pick_vertex_dense(universe)

    spec_dim = np.zeros(sigma_max)
    spec_dim1 = np.zeros(sigma_max)
    prob_buffers = [np.zeros(len(connectivity)) for _ in range(2)] 
    cur = 0
    prob_buffers[cur][start] = 1.0 
    epsilon = 1e-10  # small value to avoid numerical instability
    prob = np.zeros(sigma_max)

    # Main loop over sigma values
    for sigma in range(sigma_max):
        prob[sigma] = prob_buffers[cur][start]  # Store probability for current sigma

        # Update probability buffers for each vertex
        for vertex_id, neighbors in connectivity.items():
            # Make sure that the probability is not too small
            if prob_buffers[cur][vertex_id] > epsilon:
                # Update probability for each neighbor
                for neighbor_id in neighbors:
                    # If this vertex could not have been reached by diffusion in half of the time,
                    # there is no probability to return to the starting vertex
                    if sigma > sigma_max / 2 and prob_buffers[cur][neighbor_id] < epsilon:
                        continue
                    
                    # Update probability
                    prob_buffers[(cur + 1) % 2][neighbor_id] += (diffusion_constant *
                                                                prob_buffers[cur][vertex_id] / len(neighbors))
            
            # Prevent oscillations at low sigma (diffusion particles can sometimes remain where they are)
            prob_buffers[(cur + 1) % 2][vertex_id] += ((1.0 - diffusion_constant) *
                                                        prob_buffers[cur][vertex_id])

        # Reset current buffer
        for i in range(len(connectivity)):
            prob_buffers[cur][i] = 0.0

        cur = (cur + 1) % 2

    # Update spectral dimension
    for sigma in range(2, sigma_max - 1):
        spec_dim[sigma] = -2.0 * sigma * (prob[sigma + 1] / prob[sigma] - 1.0)
        spec_dim1[sigma] = -2.0 * np.log(prob[sigma + 1] / prob[sigma]) / np.log1p(1.0 / sigma)

    # Check if directory exists
    if not os.path.exists(f'thermal_{target_volume}/T{T}/prob'):
        os.makedirs(f'thermal_{target_volume}/T{T}/prob')
    if not os.path.exists(f'thermal_{target_volume}/T{T}/specdim'):
        os.makedirs(f'thermal_{target_volume}/T{T}/specdim')
    if not os.path.exists(f'thermal_{target_volume}/T{T}/specdim1'):
        os.makedirs(f'thermal_{target_volume}/T{T}/specdim1')

    if only_spatial:
        sptl = 0
    else:
        sptl = 1

    output_prob = f'thermal_{target_volume}/T{T}/prob/prob_sptl={sptl}_strt={start_flag}_T{T}_N{target_volume}_k0={k0}_tswps={thermal_sweeps}_swps={main_sweeps}_chain={chain}_run_{run}.txt'
    output_spec_dim = f'thermal_{target_volume}/T{T}/specdim/specdim_sptl={sptl}_strt={start_flag}_T{T}_N{target_volume}_k0={k0}_tswps={thermal_sweeps}_swps={main_sweeps}_chain={chain}_run_{run}.txt'
    output_spec_dim1 = f'thermal_{target_volume}/T{T}/specdim1/specdim1_sptl={sptl}_strt={start_flag}_T{T}_N{target_volume}_k0={k0}_tswps={thermal_sweeps}_swps={main_sweeps}_chain={chain}_run_{run}.txt'

    # Save results
    np.savetxt(output_prob, prob)
    np.savetxt(output_spec_dim, spec_dim)
    np.savetxt(output_spec_dim1, spec_dim1)

    return prob, spec_dim, spec_dim1

## Measuring

In [7]:
target_volume = 3000
k0 = 1.0
thermal_sweeps = 1000
main_sweeps = 0
diffusion_constant = 0.8
sigma_max = 1000
runs = 10
chain = 1
step = 8
chain_values_1 = np.arange(1, 73 + step, step)
print(chain_values_1)

### T3

In [9]:
# Load universe
filename3 = f'../experiments/thermal_{target_volume}/T3/saved_universes/k0={k0}/T3_k0={k0}_tswps=1000_swps=0_kstps={target_volume * 100}_chain={chain}_thermal_1000.txt'
universe3 = Universe(geometry_infilename=filename3)
universe3.update_vertices()

n0: 1490
n3: 11058
Read and loaded ../experiments/thermal_3000/T3/saved_universes/k0=1.0/T3_k0=1.0_tswps=1000_swps=0_kstps=300000_chain=1_thermal_1000.txt.


In [10]:
# only on spatial slice
for i in range(runs):
    measure_spectral_dimension(
        universe=universe3,
        sigma_max=sigma_max,
        diffusion_constant=diffusion_constant,
        only_spatial=True,
        start_flag=2,
        T=3,
        k0=k0,
        target_volume=target_volume,
        thermal_sweeps=thermal_sweeps,
        main_sweeps=main_sweeps,
        chain=chain,
        run=i
    )

In [11]:
# On whole universe
for i in range(runs):
    measure_spectral_dimension(
        universe=universe3,
        sigma_max=sigma_max,
        diffusion_constant=diffusion_constant,
        only_spatial=False,
        start_flag=2,
        T=3,
        k0=k0,
        target_volume=target_volume,
        thermal_sweeps=thermal_sweeps,
        main_sweeps=main_sweeps,
        chain=chain,
        run=i
    )

### T5

In [12]:
# Load universe
filename5 = f'../experiments/thermal_{target_volume}/T5/saved_universes/k0={k0}/T5_k0={k0}_tswps=1000_swps=0_kstps={target_volume * 100}_chain={chain}_thermal_1000.txt'
universe5 = Universe(geometry_infilename=filename5)
universe5.update_vertices()

n0: 1535
n3: 11461
Read and loaded ../experiments/thermal_3000/T5/saved_universes/k0=1.0/T5_k0=1.0_tswps=1000_swps=0_kstps=300000_chain=1_thermal_1000.txt.


In [13]:
# only on spatial slice
for i in range(runs):
    measure_spectral_dimension(
        universe=universe5,
        sigma_max=sigma_max,
        diffusion_constant=diffusion_constant,
        only_spatial=True,
        start_flag=2,
        T=5,
        k0=k0,
        target_volume=target_volume,
        thermal_sweeps=thermal_sweeps,
        main_sweeps=main_sweeps,
        chain=chain,
        run=i
    )

In [14]:
# On whole universe
for i in range(runs):
    measure_spectral_dimension(
        universe=universe5,
        sigma_max=sigma_max,
        diffusion_constant=diffusion_constant,
        only_spatial=False,
        start_flag=2,
        T=5,
        k0=k0,
        target_volume=target_volume,
        thermal_sweeps=thermal_sweeps,
        main_sweeps=main_sweeps,
        chain=chain,
        run=i
    )

### T10

In [15]:
# Load universe
filename10 = f'../experiments/thermal_{target_volume}/T10/saved_universes/k0={k0}/T10_k0={k0}_tswps=1000_swps=0_kstps={target_volume * 100}_chain={chain}_thermal_1000.txt'
universe10 = Universe(geometry_infilename=filename10)
universe10.update_vertices()

n0: 1541
n3: 11291
Read and loaded ../experiments/thermal_3000/T10/saved_universes/k0=1.0/T10_k0=1.0_tswps=1000_swps=0_kstps=300000_chain=1_thermal_1000.txt.


In [16]:
# only on spatial slice
for i in range(runs):
    measure_spectral_dimension(
        universe=universe10,
        sigma_max=sigma_max,
        diffusion_constant=diffusion_constant,
        only_spatial=True,
        start_flag=2,
        T=10,
        k0=k0,
        target_volume=target_volume,
        thermal_sweeps=thermal_sweeps,
        main_sweeps=main_sweeps,
        chain=chain,
        run=i
    )

In [17]:
# On whole universe
for i in range(runs):
    measure_spectral_dimension(
        universe=universe10,
        sigma_max=sigma_max,
        diffusion_constant=diffusion_constant,
        only_spatial=False,
        start_flag=2,
        T=10,
        k0=k0,
        target_volume=target_volume,
        thermal_sweeps=thermal_sweeps,
        main_sweeps=main_sweeps,
        chain=chain,
        run=i
    )

### Brute force run

In [20]:
target_volume = 3000
k0 = 1.0
thermal_sweeps = 1000
main_sweeps = 0
diffusion_constant = 0.8
sigma_max = 1000
runs = 10

step = 8
chain_values_1 = np.arange(9, 121 + step, step)
print(chain_values_1)

[  9  17  25  33  41  49  57  65  73  81  89  97 105 113 121]


In [21]:
for chain in chain_values_1:
    # Load universe
    filename3 = f'../experiments/thermal_{target_volume}/T3/saved_universes/k0={k0}/T3_k0={k0}_tswps=1000_swps=0_kstps={target_volume * 100}_chain={chain}_thermal_1000.txt'
    universe3 = Universe(geometry_infilename=filename3)
    universe3.update_vertices()

    # only on spatial slice
    for i in range(runs):
        measure_spectral_dimension(
            universe=universe3,
            sigma_max=sigma_max,
            diffusion_constant=diffusion_constant,
            only_spatial=True,
            start_flag=2,
            T=3,
            k0=k0,
            target_volume=target_volume,
            thermal_sweeps=thermal_sweeps,
            main_sweeps=main_sweeps,
            chain=chain,
            run=i
        )

    # On whole universe
    for i in range(runs):
        measure_spectral_dimension(
            universe=universe3,
            sigma_max=sigma_max,
            diffusion_constant=diffusion_constant,
            only_spatial=False,
            start_flag=2,
            T=3,
            k0=k0,
            target_volume=target_volume,
            thermal_sweeps=thermal_sweeps,
            main_sweeps=main_sweeps,
            chain=chain,
            run=i
        )
    
    print(f'T3 chain={chain} done')

    # Load universe
    filename5 = f'../experiments/thermal_{target_volume}/T5/saved_universes/k0={k0}/T5_k0={k0}_tswps=1000_swps=0_kstps={target_volume * 100}_chain={chain}_thermal_1000.txt'
    universe5 = Universe(geometry_infilename=filename5)
    universe5.update_vertices()

    # only on spatial slice
    for i in range(runs):
        measure_spectral_dimension(
            universe=universe5,
            sigma_max=sigma_max,
            diffusion_constant=diffusion_constant,
            only_spatial=True,
            start_flag=2,
            T=5,
            k0=k0,
            target_volume=target_volume,
            thermal_sweeps=thermal_sweeps,
            main_sweeps=main_sweeps,
            chain=chain,
            run=i
        )

    # On whole universe
    for i in range(runs):
        measure_spectral_dimension(
            universe=universe5,
            sigma_max=sigma_max,
            diffusion_constant=diffusion_constant,
            only_spatial=False,
            start_flag=2,
            T=5,
            k0=k0,
            target_volume=target_volume,
            thermal_sweeps=thermal_sweeps,
            main_sweeps=main_sweeps,
            chain=chain,
            run=i
        )

    print(f'T5 chain={chain} done')

    # Load universe
    filename10 = f'../experiments/thermal_{target_volume}/T10/saved_universes/k0={k0}/T10_k0={k0}_tswps=1000_swps=0_kstps={target_volume * 100}_chain={chain}_thermal_1000.txt'
    universe10 = Universe(geometry_infilename=filename10)
    universe10.update_vertices()

    # only on spatial slice
    for i in range(runs):
        measure_spectral_dimension(
            universe=universe10,
            sigma_max=sigma_max,
            diffusion_constant=diffusion_constant,
            only_spatial=True,
            start_flag=2,
            T=10,
            k0=k0,
            target_volume=target_volume,
            thermal_sweeps=thermal_sweeps,
            main_sweeps=main_sweeps,
            chain=chain,
            run=i
        )
    
    # On whole universe
    for i in range(runs):
        measure_spectral_dimension(
            universe=universe10,
            sigma_max=sigma_max,
            diffusion_constant=diffusion_constant,
            only_spatial=False,
            start_flag=2,
            T=10,
            k0=k0,
            target_volume=target_volume,
            thermal_sweeps=thermal_sweeps,
            main_sweeps=main_sweeps,
            chain=chain,
            run=i
        )

    print(f'T10 chain={chain} done')


n0: 1468
n3: 10953
Read and loaded ../experiments/thermal_3000/T3/saved_universes/k0=1.0/T3_k0=1.0_tswps=1000_swps=0_kstps=300000_chain=9_thermal_1000.txt.
T3 chain=9 done
n0: 1573
n3: 11593
Read and loaded ../experiments/thermal_3000/T5/saved_universes/k0=1.0/T5_k0=1.0_tswps=1000_swps=0_kstps=300000_chain=9_thermal_1000.txt.
T5 chain=9 done
n0: 1517
n3: 11067
Read and loaded ../experiments/thermal_3000/T10/saved_universes/k0=1.0/T10_k0=1.0_tswps=1000_swps=0_kstps=300000_chain=9_thermal_1000.txt.
T10 chain=9 done
n0: 1491
n3: 11018
Read and loaded ../experiments/thermal_3000/T3/saved_universes/k0=1.0/T3_k0=1.0_tswps=1000_swps=0_kstps=300000_chain=17_thermal_1000.txt.
T3 chain=17 done
n0: 1528
n3: 11367
Read and loaded ../experiments/thermal_3000/T5/saved_universes/k0=1.0/T5_k0=1.0_tswps=1000_swps=0_kstps=300000_chain=17_thermal_1000.txt.
T5 chain=17 done
n0: 1535
n3: 11209
Read and loaded ../experiments/thermal_3000/T10/saved_universes/k0=1.0/T10_k0=1.0_tswps=1000_swps=0_kstps=300000_c

KeyboardInterrupt: 